In [99]:
import isb
import numpy as np
import pandas as pd
import datetime
from scipy import optimize

In [100]:
df_index = pd.read_csv("data/index-data.csv")
df_rates = pd.read_csv('data/base-rates.csv')
df_salary_numbers = pd.read_csv('data/salary-numbers.csv')

In [101]:
df = df_index.merge(df_rates, on='date')\
.merge(df_salary_numbers, on='date', how='left')


In [102]:
first_tax_row = df.loc[lambda r: r.date == '2016-01-01'].iloc[0]

df.loc[df['brutto_salary'].isnull(), ['brutto_salary']] = df.loc[df['brutto_salary'].isnull(), 'salary']
df.loc[df['mandatory_pension'].isnull(), ['mandatory_pension']] = \
    df.loc[df['mandatory_pension'].isnull(), 'brutto_salary']*first_tax_row.mandatory_pension/first_tax_row.brutto_salary
df.loc[df['private_pension'].isnull(), ['private_pension']] = \
    df.loc[df['private_pension'].isnull(), 'brutto_salary']*first_tax_row.private_pension/first_tax_row.brutto_salary
df.loc[df['netto_salary'].isnull(), ['netto_salary']] = \
    df.loc[df['netto_salary'].isnull(), 'brutto_salary']*first_tax_row.netto_salary/first_tax_row.brutto_salary

df['salary'] = df['netto_salary']*2

In [117]:
df.loc[df.date == '2025-10-01', ['nonindex-a-fix-5y']] = 0.0795
df.loc[df.date == '2025-10-01', ['index-a-fix-5y']] = 0.0455

In [118]:
df

,Unnamed: 0,date,salary_index,value_12m_change,property_index,salary,salary_yearly,property,finance_index,index-a-fix-5y,...,taxable_salary,personal_discount,non_index_rate,index_rate,non_index_duration,index_duration,finance_cost_indexed,finance_cost_non_indexed,fci_indexed,fci_non_indexed
0,324,2016-01-01,545.0,0.094158,45.2486,643360.0,5.501087e+06,4.057619e+07,7.376032,0.0395,...,430917.0,51920.0,0.0725,0.0395,40,40,117826.731688,181688.146988,0.183143,0.282405
1,325,2016-02-01,564.0,0.126198,45.4290,661884.0,5.692868e+06,4.073796e+07,7.155965,0.0395,...,445940.0,51920.0,0.0725,0.0395,40,40,118296.490805,182412.512862,0.178727,0.275596
2,326,2016-03-01,568.8,0.132842,45.7598,666564.0,5.741318e+06,4.103460e+07,7.147245,0.0395,...,449736.0,51920.0,0.0725,0.0395,40,40,119157.889453,183740.784655,0.178764,0.275654
3,327,2016-04-01,570.4,0.133545,46.0906,668122.0,5.757468e+06,4.133124e+07,7.178719,0.0385,...,451000.0,51920.0,0.0725,0.0385,40,40,118232.450308,185069.056447,0.176962,0.276999
4,328,2016-05-01,573.1,0.133281,46.6219,670756.0,5.784721e+06,4.180768e+07,7.227260,0.0385,...,453136.0,51920.0,0.0725,0.0385,40,40,119595.350787,187202.402285,0.178299,0.279092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,437,2025-06-01,1088.8,0.081554,111.4000,1266414.0,1.099006e+07,9.989674e+07,9.089735,0.0480,...,860887.0,68691.0,0.0800,0.0480,40,25,400683.696295,486215.593414,0.316392,0.383931
114,438,2025-07-01,1086.3,0.076397,111.7000,1263962.0,1.096483e+07,1.001658e+08,9.135189,0.0475,...,858910.0,68691.0,0.0800,0.0475,40,25,399743.675014,487524.971135,0.316262,0.385712
115,439,2025-08-01,1087.7,0.076931,111.1000,1265336.0,1.097896e+07,9.962772e+07,9.074424,0.0475,...,860018.0,68691.0,0.0800,0.0475,40,25,397596.439517,484906.215694,0.314222,0.383223
116,440,2025-09-01,1096.4,0.077967,112.8000,1273868.0,1.106677e+07,1.011522e+08,9.140169,0.0475,...,866897.0,68691.0,0.0800,0.0475,40,25,403680.273425,492326.022775,0.316893,0.386481


In [119]:
def monthly_payment(loan_amount, rate, years=40):
    rate = rate/12
    n = years*12
    loan_amount = loan_amount*.7
    return loan_amount*((1+rate)**n) * rate/((1 + rate)**n - 1)

def monthly_payment_proxy(row):
    return monthly_payment(row[0], row[1], row[2])

def finance_cost(p, r, s, y=40):
    return monthly_payment(p, r, years=y)/s

In [123]:
df = df\
.assign(
    non_index_rate = lambda r: r[['nonindex-a-fix-5y', 'nonindex-a-float', 'nonindex-a-fix-3y']].min(axis=1),
    index_rate = lambda r: r[['index-a-fix-5y', 'index-a-float']].min(axis=1),
    non_index_duration = 40,
    index_duration = lambda r: r.apply(lambda x: 25 if x.date >= '2023-01-01' else 40, axis=1),
)\
.assign(
    finance_cost_indexed = lambda r: r[['property', 'index_rate', 'index_duration']].apply(monthly_payment_proxy, axis=1),
    finance_cost_non_indexed = lambda r: r[['property', 'non_index_rate', 'non_index_duration']].apply(monthly_payment_proxy, axis=1),
    #finance_cost = lambda r: r.finance_cost_indexed * r.index_pct + r.finance_cost_non_indexed * r.non_index_pct
    fci_indexed = lambda r: r.finance_cost_indexed / r.salary,
    fci_non_indexed = lambda r: r.finance_cost_non_indexed / r.salary,
    #fci = lambda r: r.finance_cost / r.salary
)

df.tail()

C:\Users\palmarg\AppData\Local\Temp\ipykernel_34604\1205500066.py:8: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\palmarg\AppData\Local\Temp\ipykernel_34604\1205500066.py:8: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



,Unnamed: 0,date,salary_index,value_12m_change,property_index,salary,salary_yearly,property,finance_index,index-a-fix-5y,...,taxable_salary,personal_discount,non_index_rate,index_rate,non_index_duration,index_duration,finance_cost_indexed,finance_cost_non_indexed,fci_indexed,fci_non_indexed
113,437,2025-06-01,1088.8,0.081554,111.4,1266414.0,1.099006e+07,9.989674e+07,9.089735,0.0480,...,860887.0,68691.0,0.0800,0.0480,40,25,400683.696295,486215.593414,0.316392,0.383931
114,438,2025-07-01,1086.3,0.076397,111.7,1263962.0,1.096483e+07,1.001658e+08,9.135189,0.0475,...,858910.0,68691.0,0.0800,0.0475,40,25,399743.675014,487524.971135,0.316262,0.385712
115,439,2025-08-01,1087.7,0.076931,111.1,1265336.0,1.097896e+07,9.962772e+07,9.074424,0.0475,...,860018.0,68691.0,0.0800,0.0475,40,25,397596.439517,484906.215694,0.314222,0.383223
116,440,2025-09-01,1096.4,0.077967,112.8,1273868.0,1.106677e+07,1.011522e+08,9.140169,0.0475,...,866897.0,68691.0,0.0800,0.0475,40,25,403680.273425,492326.022775,0.316893,0.386481
117,441,2025-10-01,1098.1,0.076569,112.8,1275534.0,1.108393e+07,1.011522e+08,9.126018,0.0455,...,868241.0,68691.0,0.0795,0.0455,40,25,395577.839287,489671.186560,0.310127,0.383895


In [131]:
fig_colors = isb.Figure()

def make_line_plot(df, fig):

    for i, column in enumerate(df.columns):

        fig.add_scatter(
            x = df.index,
            y = df[column],
            mode = 'lines',
            line = dict(
                color = fig_colors.colors(i)
            ),
            row=1,
            col=i+1,
            showlegend=False
        )


fig = isb.Figure('core')
fig.set_title(
    "Hlutfall ráðstöfunartekna sem fer í afborgun húsnæðisláns",
    subtitle="Miðað við tvo einstaklinga á meðallaunum og 70% lán á 120fm eign á höfuðborgarsvæðinu"
)
fig.as_subplot(rows=1, cols=2, subplot_titles=["<b>Verðtryggt húsnæðislán</b>", "<b>Óverðtryggt húsnæðislán</b>"],
                shared_yaxes=True, horizontal_spacing=.05)


df\
.set_index('date')\
.loc[lambda r: r.index >= '2016-01-01']\
.loc[:, ['fci_indexed', 'fci_non_indexed']]\
.pipe(make_line_plot, fig=fig)

fig.update_layout(
    yaxis = dict(
        tickformat = '.0%'
    )
)

fig.add_hline(
    y = .35,
    line = dict(
        dash = 'dash',
        color = isb.ph.hex_to_rgba("#333333", .6)
    )
)


annotations = []
for i, annotation in enumerate(fig.layout.annotations):
    if "tryggt" in annotation['text']:
        annotation['font'] = fig.get_font(size = 24, color = fig_colors.colors(i))
        annotation['y'] -= .02
        annotation['bgcolor'] = fig.bg_color

fig.add_annotation(
    x = '2020-06-01',
    y = .35,
    text = 'Hámarks greiðslubyrði fasteignalána',
    showarrow = False,
    yanchor='bottom',
    font = fig.get_font(size = 16)
)

fig.export('DSI', scale=1.5)
fig.show()

In [71]:
def create_salary_to_price(theme):
    def make_line_plot(df, fig):

        for i, column in enumerate(df.columns):

            fig.add_scatter(
                x = df.index,
                y = df[column],
                mode = 'lines',
                line = dict(
                    color = fig.colors(i)
                ),
                showlegend=False
            )

    fig = delta.Figure("Fasteignaverð sem hlutfall af árslaunum", subtitle="Miðað við 120fm íbúð á höfuðborgarsvæðinu",
                       theme = theme)

    df\
    .assign(
        property_to_salary = lambda r: r.property / r.salary_yearly
    )\
    .set_index('date')\
    .loc[:, ['property_to_salary']]\
    .pipe(make_line_plot, fig=fig)

    fig.update_layout(
        xaxis = dict(
            dtick = 'M24'
        ),
        yaxis = dict(
            tickformat = '.1f',
            ticksuffix = ' ',
            title = '<b></b>',
            titlefont = fig.get_font(size=20)
        )
    )

    fig.add_annotation(
        xref = 'paper', yref = 'paper',
        x = 1.00, y = -0.08,
        text = 'Heimildir: vísitala íbúðaverðs frá HMS, launavísitala frá Hagstofu Íslands',
        showarrow = False,
        yanchor='bottom',
        font = fig.get_font(size = 16)
    )

    return fig



for theme in [theme_innherji, theme_vidskiptabladid, theme_vidskiptabladid_no_bg, theme_base]:
    fig = create_salary_to_price(theme)
    fig.export("salary-to-price", write_svg=True)


fig.show()

NameError: name 'theme_innherji' is not defined

In [13]:
#fci_stability = df.loc[lambda r: r.date.year.isin(range(2011, 2020)), ['fci_indexed', 'fci_non_indexed']].mean(axis=0)
#fci_stability

fci_stability = dict(
    fci_indexed = .183,
    fci_non_indexed = .269,
    fci = .211
)

latest_row = df.iloc[-1]
latest_row

stability_data = []

def get_property_objective(stability_point, rate_name):
    def property_objective(x):
        return 1e6*abs(finance_cost(x, latest_row[rate_name], latest_row.salary) - stability_point)
    return property_objective

for n, s in fci_stability.items():

    if n == 'fci':
        continue

    print(n)
    n = n.replace('fci_', '')

    if n == 'indexed':
        n = 'index_rate'
    else:
        n = 'non_index_rate'

    opt = optimize.minimize(
        get_property_objective(s, n),
        latest_row.property,
        method='Nelder-Mead',
        options={ 'maxiter': 1000 }
    )
    print(opt)
    print(f"Fasteignaverð þarf að hækka um {100*round(opt.x[0]/latest_row.property - 1, 3)}%")

    stability_data.append({
        'category': 'property',
        'rate_type': n,
        'changes_needed': opt.x[0]/latest_row.property - 1
    })

fci_indexed
 final_simplex: (array([[92827500.57673763],
       [92827500.57666427]]), array([6.99440506e-09, 1.37639899e-07]))
           fun: 6.994405055138486e-09
       message: 'Optimization terminated successfully.'
          nfev: 78
           nit: 39
        status: 0
       success: True
             x: array([92827500.57673763])
Fasteignaverð þarf að hækka um -7.9%
fci_non_indexed
 final_simplex: (array([[72776461.22789055],
       [72776461.2278172 ]]), array([7.72715225e-08, 1.93900451e-07]))
           fun: 7.72715225139109e-08
       message: 'Optimization terminated successfully.'
          nfev: 82
           nit: 41
        status: 0
       success: True
             x: array([72776461.22789055])
Fasteignaverð þarf að hækka um -27.800000000000004%


In [14]:
df[df.finance_index >= 8.955267]\
.loc[:, ['date', 'finance_index']]

,date,finance_index
125,2007-10-01,8.955267
302,2022-07-01,9.040705
303,2022-08-01,8.983882
304,2022-09-01,8.984543
305,2022-10-01,9.006091


In [15]:
8.955267

8.955267

In [16]:
finance_cost_index_stability = 0.269

def get_property_objective(r, s):
    def property_objective(x):
        return 1e6*abs(finance_cost(x, latest_row.non_index_rate + r, latest_row.salary*(1+s)) - finance_cost_index_stability)
    return property_objective

rate_salary = []
for rate in range(-150, 1, 10):
    rate = rate/100

    for salary in range(0, 160, 10):
        salary = salary/1000

        opt = optimize.minimize(
            get_property_objective(rate, salary), latest_row.property,
            method='Nelder-Mead',
            options={'maxiter':1000}
        )

        rate_salary.append({
            'rate_increase': rate/100,
            'rate': latest_row.non_index_rate,
            'salary_increase': salary,
            'salary': latest_row.salary * (1 + salary),
            'price_increase': opt.x[0]/latest_row.property - 1
        })

In [17]:
df_z = pd.DataFrame(rate_salary)\

df_z_pivot = df_z\
.pivot(index='rate_increase', columns='salary_increase', values='price_increase')

df_z

,rate_increase,rate,salary_increase,salary,price_increase
0,-0.015,7.59,0.00,1258876.00,-0.137815
1,-0.015,7.59,0.01,1271464.76,-0.129193
2,-0.015,7.59,0.02,1284053.52,-0.120571
3,-0.015,7.59,0.03,1296642.28,-0.111949
4,-0.015,7.59,0.04,1309231.04,-0.103327
...,...,...,...,...,...
251,0.000,7.59,0.11,1397352.36,-0.198797
252,0.000,7.59,0.12,1409941.12,-0.191579
253,0.000,7.59,0.13,1422529.88,-0.184361
254,0.000,7.59,0.14,1435118.64,-0.177142


In [18]:
def create_sensitivity_plot(theme):

    fig = delta.Figure("Áhrif launahækkana og vaxtastigs á fasteignaverð",
                       subtitle="Talan innan kassana sýnir breytingu á fasteignaverði, miðað við forsendur ásana",
                       theme=theme, height=1920, width=1920)

    scaler = ph.create_max_min_color_scaler(df_z.price_increase, color_low='#ffffff', color_high="#eeeeee")

    fig.add_heatmap(
        z = df_z_pivot.values,
        x = df_z_pivot.columns,
        y = df_z_pivot.index,
        colorscale = 'Magma',
        showscale = False,
    )

    for i, row in df_z.iterrows():

        color = scaler(row.price_increase)
        if row.price_increase > -.12:
            color = fig.font_color

        fig.add_annotation(
            x = row.salary_increase,
            y = row.rate_increase,
            text = ph.bold(str(round(row.price_increase*100, 1)) + '%'),
            showarrow = False,
            font = fig.get_font(size = 18, color = color)
        )


    fig.update_xaxes(
        tickformat = '.0%',
        dtick = .01,
        title = "Prósentu hækkun launa",
        tickfont = fig.get_font(
            size = 20
        ),
        titlefont = fig.get_font(
            size = 24
        )
    )

    fig.update_xaxes(
        titlefont = fig.get_font(
            size = 24
        )
    )

    fig.update_yaxes(
        tickformat = '.1%',
        dtick = .001,
        title = "Prósentustigs lækkun óverðtryggðra húsnæðisvaxta",
        tickfont = fig.get_font(
            size = 20
        ),
        titlefont = fig.get_font(
            size = 24
        )
    )

    fig.update_yaxes(
        titlefont = fig.get_font(
            size = 24
        )
    )

    return fig



for theme in [theme_innherji, theme_vidskiptabladid, theme_vidskiptabladid_no_bg, theme_base]:
    fig = create_sensitivity_plot(theme)
    fig.export("sensitivity", write_svg=True)


fig.show()


In [19]:
data = """Jan-13	Feb-13	Mar-13	Apr-13	May-13	Jun-13	Jul-13	Aug-13	Sep-13	Oct-13	Nov-13	Dec-13	Jan-14	Feb-14	Mar-14	Apr-14	May-14	Jun-14	Jul-14	Aug-14	Sep-14	Oct-14	Nov-14	Dec-14	Jan-15	Feb-15	Mar-15	Apr-15	May-15	Jun-15	Jul-15	Aug-15	Sep-15	Oct-15	Nov-15	Dec-15	Jan-16	Feb-16	Mar-16	Apr-16	May-16	Jun-16	Jul-16	Aug-16	Sep-16	Oct-16	Nov-16	Dec-16	Jan-17	Feb-17	Mar-17	Apr-17	May-17	Jun-17	Jul-17	Aug-17	Sep-17	Oct-17	Nov-17	Dec-17	Jan-18	Feb-18	Mar-18	Apr-18	May-18	Jun-18	Jul-18	Aug-18	Sep-18	Oct-18	Nov-18	Dec-18	Jan-19	Feb-19	Mar-19	Apr-19	May-19	Jun-19	Jul-19	Aug-19	Sep-19	Oct-19	Nov-19	Dec-19	Jan-20	Feb-20	Mar-20	Apr-20	May-20	Jun-20	Jul-20	Aug-20	Sep-20	Oct-20	Nov-20	Dec-20	Jan-21	Feb-21	Mar-21	Apr-21	May-21	Jun-21	Jul-21	Aug-21	Sep-21	Oct-21	Nov-21	Dec-21	Jan-22	Feb-22	Mar-22	Apr-22	May-22	Jun-22	Jul-22	Aug-22	Sep-22	Oct-22	Nov-22
898	542	646	912	1,304	841	1,149	1,443	1,222	1,129	741	588	743	184	647	474	464	381	496	410	547	1,117	848	86	-781	97	815	296	19	154	1,381	255	775	0	-1,043	-633	-942	-819	-1,528	-1,475	-1,119	-592	-1,217	-687	-1,121	-480	-176	-329	132	148	316	79	998	642	1,278	1,547	1,470	1,857	3,032	2,905	2,392	1,996	2,257	2,566	3,127	2,940	2,716	2,933	3,237	2,401	1,558	1,277	1,553	3,346	2,900	3,556	5,238	4,930	7,481	6,508	7,350	10,167	10,745	7,999	9,567	7,929	12,920	17,447	27,433	31,038	44,924	40,965	38,770	39,219	25,185	20,071	24,833	22,325	25,953	29,635	29,960	17,892	11,637	8,788	9,006	6,758	5,048	1,759	4,601	-1,193	-4,822	877	4,408	2,167	4,254	4,512	1,855	1,757	965
1,513	933	1,243	1,354	1,085	1,474	1,617	1,022	1,090	1,367	931	1,028	1,131	717	880	552	559	832	866	772	902	790	991	267	-457	324	812	431	199	1,648	4,732	5,797	5,293	3,300	4,211	3,343	927	1,186	550	1,158	1,045	596	1,022	626	460	72	-300	219	110	129	240	128	-498	-129	688	1,195	2,016	2,897	1,458	1,524	1,478	1,552	2,857	2,066	3,495	2,776	2,895	2,575	4,264	7,862	12,723	9,370	8,510	3,806	3,750	3,730	2,012	-189	-191	-396	-268	-1,650	-2,642	-1,693	-746	-1,283	-2,230	-2,767	-3,354	-2,556	391	2,192	5,098	16,959	20,733	15,802	10,581	4,347	5,516	6,277	9,483	19,471	24,463	13,797	20,249	16,941	18,272	19,614	8,272	12,919	21,444	9,967	11,599	13,520	11,346	6,332	4,033	1,073	158
470	665	642	761	1,288	992	1,177	1,185	1,576	1,201	1,093	1,014	731	981	1,284	1,183	1,793	1,813	1,991	1,831	2,112	2,665	1,865	1,907	1,663	1,868	3,035	1,235	706	1,108	4,373	2,599	3,497	2,881	2,970	2,846	2,256	2,810	2,072	3,191	2,988	3,257	3,583	3,678	3,862	4,796	5,936	6,255	5,650	5,249	7,969	4,367	6,753	7,585	7,166	6,567	5,440	6,410	4,601	4,895	3,848	3,567	3,692	2,990	3,734	4,367	5,675	3,728	3,989	896	-1,152	5	-351	233	1,328	394	2,208	2,916	2,354	3,411	3,299	4,068	3,919	2,460	2,000	2,183	1,627	-549	177	413	79	-2,205	-4,351	-5,335	-6,591	-4,595	-4,004	-2,960	-3,993	-2,710	-4,388	-6,218	-5,262	-3,501	-4,538	-3,873	-4,043	-2,969	-1,456	-2,684	-4,262	-2,955	-3,867	-4,122	-2,840	1,352	634	3,115	4,290
306	96	-12	179	568	721	350	427	640	497	169	6	-838	-377	248	229	-218	329	1,121	729	800	678	1,458	1,532	-2,425	-22	1,184	58	-124	344	3,717	2,998	4,218	3,792	3,841	2,826	-969	2,387	1,982	2,257	3,288	2,957	2,637	3,179	2,817	2,483	2,242	3,529	1,540	1,864	2,138	2,189	520	156	547	681	-657	70	-583	-462	245	-784	1,036	556	4	-950	291	46	446	-128	-1,982	-1,277	-1,184	-368	149	1,091	624	-67	317	333	116	-1,135	-2,578	-2,200	-1,668	-1,444	-1,178	-2,287	-4,047	-5,486	-7,594	-6,145	-4,598	-4,859	-2,649	1,163	-841	-598	-861	-442	-1,448	-961	-127	-472	-1,132	-530	353	150	1,885	652	3,468	3,967	4,919	5,954	4,828	4,022	5,068	4,747	2,887"""

In [20]:
def convert_date(date):

    month, year = date.split("-")
    month = dict(zip(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], range(1, 13)))[month]
    return datetime.date(int(f"20{year}"), int(month), 1)

df_loans = pd.DataFrame([_.split("\t") for _ in data.split("\n")])\
.T\
.rename(columns={
    0: 'date',
    1: 'non_indexed_float',
    2: 'non_indexed_fixed',
    3: 'indexed_float',
    4: 'indexed_fixed'
})\
.assign(
    date = lambda r: r.date.apply(convert_date).pipe(pd.to_datetime),
    non_indexed_float = lambda r: r.non_indexed_float.str.replace(",", "").astype(int) * 1e6,
    non_indexed_fixed = lambda r: r.non_indexed_fixed.str.replace(",", "").astype(int) * 1e6,
    indexed_float = lambda r: r.indexed_float.str.replace(",", "").astype(int) * 1e6,
    indexed_fixed = lambda r: r.indexed_fixed.str.replace(",", "").astype(int) * 1e6
)

In [21]:
data_pension = """Jan-09	Feb-09	Mar-09	Apr-09	May-09	Jun-09	Jul-09	Aug-09	Sep-09	Oct-09	Nov-09	Dec-09	Jan-10	Feb-10	Mar-10	Apr-10	May-10	Jun-10	Jul-10	Aug-10	Sep-10	Oct-10	Nov-10	Dec-10	Jan-11	Feb-11	Mar-11	Apr-11	May-11	Jun-11	Jul-11	Aug-11	Sep-11	Oct-11	Nov-11	Dec-11	Jan-12	Feb-12	Mar-12	Apr-12	May-12	Jun-12	Jul-12	Aug-12	Sep-12	Oct-12	Nov-12	Dec-12	Jan-13	Feb-13	Mar-13	Apr-13	May-13	Jun-13	Jul-13	Aug-13	Sep-13	Oct-13	Nov-13	Dec-13	Jan-14	Feb-14	Mar-14	Apr-14	May-14	Jun-14	Jul-14	Aug-14	Sep-14	Oct-14	Nov-14	Dec-14	Jan-15	Feb-15	Mar-15	Apr-15	May-15	Jun-15	Jul-15	Aug-15	Sep-15	Oct-15	Nov-15	Dec-15	Jan-16	Feb-16	Mar-16	Apr-16	May-16	Jun-16	Jul-16	Aug-16	Sep-16	Oct-16	Nov-16	Dec-16	Jan-17	Feb-17	Mar-17	Apr-17	May-17	Jun-17	Jul-17	Aug-17	Sep-17	Oct-17	Nov-17	Dec-17	Jan-18	Feb-18	Mar-18	Apr-18	May-18	Jun-18	Jul-18	Aug-18	Sep-18	Oct-18	Nov-18	Dec-18	Jan-19	Feb-19	Mar-19	Apr-19	May-19	Jun-19	Jul-19	Aug-19	Sep-19	Oct-19	Nov-19	Dec-19	Jan-20	Feb-20	Mar-20	Apr-20	May-20	Jun-20	Jul-20	Aug-20	Sep-20	Oct-20	Nov-20	Dec-20	Jan-21	Feb-21	Mar-21	Apr-21	May-21	Jun-21	Jul-21	Aug-21	Sep-21	Oct-21	Nov-21	Dec-21	Jan-22	Feb-22	Mar-22	Apr-22	May-22	Jun-22	Jul-22	Aug-22	Sep-22	Oct-22	Nov-22
683	874	1,054	966	1,000	1,177	882	614	704	609	719	563	448	532	842	530	611	837	578	528	707	545	766	729	884	747	664	904	932	920	965	916	987	490	582	611	304	353	332	199	508	623	733	885	747	830	833	790	797	627	541	692	654	744	687	820	903	1,192	574	578	737	724	809	861	1,091	1,196	1,149	1,104	896	1,287	944	857	1,064	1,262	1,686	288	120	574	2,894	1,764	1,006	475	1,371	1,089	1,567	1,615	3,160	4,409	3,718	3,854	3,949	5,054	4,256	5,763	5,968	4,437	4,772	4,510	5,758	3,665	6,214	8,002	7,251	7,387	3,287	3,951	5,737	6,451	5,838	5,072	5,788	4,737	7,910	6,700	8,076	8,278	6,544	4,519	3,425	1,977	4,319	4,060	3,162	3,685	4,845	5,252	5,118	3,919	5,669	6,875	8,215	5,407	6,923	4,326	4,258	585	202	-982	-3,863	-4,964	-5,840	-9,314	-9,544	-7,171	-4,550	-4,522	-5,470	-4,427	-6,606	-6,713	-4,532	-2,849	-2,521	-2,861	-2,275	-2,108	-892	-2,492	-2,886	-2,401	-3,878	-3,352	-2,681	-2,346	-72	1,018	1,169
																																																																										0						0	-9	0	667	1,107	1,365	1,638	1,999	1,971	2,008	1,637	1,789	1,484	1,146	1,501	1,639	2,187	2,406	3,674	1,694	2,903	3,048	2,349	2,695	1,644	3,079	3,992	2,628	1,459	1,173	2,296	1,936	2,240	2,349	2,465	1,931	3,062	3,528	3,315	3,161	4,761	3,361	2,574	2,117	3,133	2,464	2,250	2,405	2,783	7,079	4,365	3,788	4,710	3,315	2,596	894	716	573	-1,270	88	2,513	359	338	833	458	756	2,033	996	399	3,169	2,965	1,678	2,453	1,671	3,724	3,963	2,874	5,906	9,149	7,385	6,598	4,393	5,593	7,144	5,624	5,774	3,698"""

In [22]:
df_loans_pension = pd.DataFrame([[_1.strip() for _1 in _.split("\t")] for _ in data_pension.split("\n")])\
.T\
.rename(columns={
    0: 'date',
    1: 'indexed',
    2: 'non_indexed'
})\
.replace("", 0)\
.assign(
    date = lambda r: r.date.apply(convert_date).pipe(pd.to_datetime),
    indexed = lambda r: r.indexed.str.replace(",", "").fillna(0).astype(int) * 1e6,
    non_indexed = lambda r: r.non_indexed.str.replace(",", "").fillna(0).astype(int) * 1e6
)

In [23]:
name_mapping = dict(
    non_indexed_float = "Óverðtryggt breytilegir",
    non_indexed_fixed = "Óverðtryggt fastir",
    indexed_float = "Verðtryggt breytilegir",
    indexed_fixed = "Verðtryggt fastir"
)

def create_loan_composition(theme):

    def make_line_plot(df, fig):

        for i, column in enumerate(df.columns):

            fig.add_bar(
                x = df.index,
                y = df[column],
                marker = dict(
                    color = fig.colors_ibm(i)
                ),
                name = name_mapping[column],
            )

    fig = delta.Figure(
        "Ný húsnæðislán að frádregnum upp- og umframgreiðslum",
        subtitle="Gögn frá Seðlabanka Íslands, öll lán lífeyrissjóða flokkuð á breytilegum vöxtum*.",
        theme=theme
    )

    df_loans\
    .merge(df_loans_pension, on='date', how='left')\
    .assign(
            non_indexed_float = lambda r: r.non_indexed_float + r.non_indexed,
            indexed_float =lambda r: r.indexed_float + r.indexed
        )\
    .set_index("date")\
    .loc[:, ['non_indexed_float', 'non_indexed_fixed', 'indexed_float', 'indexed_fixed']]\
    .pipe(make_line_plot, fig=fig)

    fig.update_layout(
        barmode = 'relative',
        legend = dict(
            x = .33,
            y = .825,
            font = fig.get_font(size = 24)
        )
    )

    fig.add_annotation(
        xref = 'paper', yref = 'paper',
        x = 1.00, y = -0.08,
        text = '* Gögn bjóða ekki uppá skiptinguna fyrir lífeyrissjóði. Heimildir: Seðlabanki Íslands',
        showarrow = False,
        yanchor='bottom',
        font = fig.get_font(size = 16)
    )

    return fig


for theme in [theme_innherji, theme_vidskiptabladid, theme_vidskiptabladid_no_bg, theme_base]:
    fig = create_loan_composition(theme)
    fig.export("loan-composition", write_svg=True)



fig.show()

In [24]:
df_loans\
    .merge(df_loans_pension, on='date', how='left')\
    .assign(
    non_indexed_float = lambda r: r.non_indexed_float + r.non_indexed,
    indexed_float =lambda r: r.indexed_float + r.indexed
)\
.set_index("date")\
.loc[:, ['non_indexed_float', 'non_indexed_fixed', 'indexed_float', 'indexed_fixed']]\
.iloc[-1]

non_indexed_float    4.663000e+09
non_indexed_fixed    1.580000e+08
indexed_float        5.459000e+09
indexed_fixed        2.887000e+09
Name: 2022-11-01 00:00:00, dtype: float64

In [25]:
(4.663000e+09 + 1.580000e+08) / (4.663000e+09 + 1.580000e+08 + 5.459000e+09 + 2.887000e+09)

0.36614262930052405

In [26]:
df_loans\
.loc[lambda r: r.date.between(pd.to_datetime("2020-01-01"), pd.to_datetime("2020-12-01"))]\
.non_indexed_fixed.sum()

48239000000.0

In [27]:
48239000000.0 / 1e6

48239.0